In [2]:
import pandas as pd
import csv
import sys
import re
import scipy
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from time import time

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [3]:
def process_content(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()
        source_code = []
        for line in lines:
            # filter comments
            if not re.match("\s*\/\/\s*isComment", line):
                source_code.append(line)
    text = ' '.join(source_code)
    return text

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

param_dist = {'C': scipy.stats.expon(scale=100)}

lr_classifier = LogisticRegression(random_state=42)

n_iter_search = 20
random_search = RandomizedSearchCV(lr_classifier,
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   cv=5,
                                   n_jobs=-1)

In [3]:
df = pd.read_pickle('../data/instances.pkl')
labels = list(set(df['target'].values))

In [ ]:
X_train = []
Y_train = []
X_test = []
Y_test = []

print("Preparing lists...")
for index, row in df.iterrows():
    X_train.append(process_content(row["source_code"]))
    Y_train.append(row["target"])

print("Extracting features...")
cv = CountVectorizer(binary=True)
cv.fit(X_train)
train_instances = cv.transform(X_train)
test_instances = cv.transform(X_test)

X_t, X_v, y_t, y_v = train_test_split(train_instances, Y_train, train_size = 0.75)

start = time()
print("Hyperparameter tuning...")
random_search.fit(train_instances, Y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
  " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)
# print("============ EVALUATION on test set:")
# print(accuracy_score(Y_test, random_search.best_estimator_.predict(test_instances)))

Preparing lists...
Extracting features...
Hyperparameter tuning...


/Users/erickmaziero/virtualenvs/refact_env/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
